# ***Simple iteration for systems of linear equations***

First, generate a random diagonally dominant matrix, for testing.

In [ ]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [ ]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [ ]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [ ]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [ ]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [ ]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00, -2.22044605e-16, -1.11022302e-16,
        1.11022302e-16,  0.00000000e+00, -2.42861287e-17,  0.00000000e+00,
       -2.77555756e-17,  1.11022302e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [ ]:
#Python 3

#Based on the works done in Week 3

def J_iter(A, b, eps=1e-12, maxiter=50): 
    
    #Separate the diagonal part
    first_diag = np.diag(A) 
    
    B = -A.copy() #Rewrite the matrix

    np.fill_diagonal(B, 0) # to iterate we need the diagonal we just got.
    invD = np.diag(1.0 / first_diag)
    
    BB = invD @ B #got the inverse
    c = invD @ b 
    
    zero_x = np.ones(n) #then we iterate
    for i in range(maxiter): 
        x = BB @ zero_x + c 
        
        if np.linalg.norm(zero_x - x) < eps: 
            break
        
        zero_x = x
    
    return x 

Tests performed *(making use of the allclose NumPy function)*.

In [ ]:
x = J_iter(A, b)

np.testing.assert_allclose(A @ xx, b)
np.testing.assert_allclose(xx, x)

Now we define a norm function, to check the convergence.

In [ ]:
#Python 3

def norm(A):

    #Procedure displayed separately from Jacobi Function 
    diag_1d = np.diag(A)
    
    B = -A.copy()
    np.fill_diagonal(B, 0)
    invD = np.diag(1.0 / diag_1d)
    
    BB = invD @ B

    Norm = np.linalg.norm(BB)
    
    return Norm

In [ ]:
# Comparison item vs. item per matrix

for k in range(25): 
    A1 = A + np.diagflat([-k] * n)
    B_norm = norm(A1)
    x = J_iter(A1, b)
    
    xx = np.linalg.solve(A1, b)
    dist = np.linalg.norm(x - xx)
    
    print(A1[0, 0], '\t', B_norm, '\t', dist)

15.191519450378893 	 0.36436161983015336 	 8.016888858755822e-14
14.191519450378893 	 0.38959181027260875 	 1.4606182072244602e-13
13.191519450378893 	 0.4185783948614869 	 1.1984374732656778e-13
12.191519450378893 	 0.4522284025473819 	 1.555049063248564e-13
11.191519450378893 	 0.4917667095178099 	 1.376077494702953e-13
10.191519450378893 	 0.5388887887486234 	 2.6702804830291413e-13
9.191519450378893 	 0.5960110344093966 	 3.1111985906662684e-13
8.191519450378893 	 0.6667001660296402 	 2.7708368486989536e-13
7.191519450378893 	 0.7564517359241753 	 1.4990528448342257e-10
6.191519450378893 	 0.8742017351588476 	 2.016159609243833e-07
5.191519450378893 	 1.0355299928250665 	 0.0009191717405677776
4.191519450378893 	 1.2702850939751231 	 23.48163367953915
3.191519450378893 	 1.6439565658213244 	 8260242.793633645
2.191519450378893 	 2.334809111760855 	 261149802433164.72
1.1915194503788928 	 4.080768845910033 	 1.3716691464130682e+26
0.19151945037889284 	 30.715327603064885 	 1.7398635

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [ ]:
#Python 3

def GS_iter(A, b, eps=1e-15, maxiter=100):

    x = np.zeros_like(b, dtype=np.double)
    
    #Iteration
    for k in range(maxiter):
        
        x_old  = x.copy()
        
        #Loop over rows
        for i in range(A.shape[0]):
            x[i] = (b[i] - np.dot(A[i,:i], x[:i]) - np.dot(A[i,(i+1):], x_old[(i+1):])) / A[i ,i]
            
        #Stop condition 
        if np.linalg.norm(x - x_old, ord=np.inf) / np.linalg.norm(x, ord=np.inf) < eps:
            break
            
    return x  
        

In [ ]:
#Random Linear Solveig Test

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

xx = np.linalg.solve(A, b)

In [ ]:
x = GS_iter(A, b)

Tests performed *(making use of the allclose NumPy function)*.

In [ ]:
np.testing.assert_allclose(x, xx)
np.testing.assert_allclose(A @ x, b)

In [ ]:
#Python 3

for k in range(25): 
    A1 = A + np.diagflat([-k] * n)
    B_norm = norm(A1)
    x = GS_iter(A1, b)
    
    xx = np.linalg.solve(A1, b)
    dist = np.linalg.norm(x - xx)
    
    print(A1[0, 0], '\t', B_norm, '\t', dist)

15.325707414425347 	 0.3679761958864619 	 1.97026712507607e-17
14.325707414425347 	 0.39315162237575607 	 1.8440363494407353e-17
13.325707414425347 	 0.42202631543251723 	 2.3864272523720094e-17
12.325707414425347 	 0.45548069473587693 	 2.2882574235218375e-17
11.325707414425347 	 0.4946988077730063 	 2.124593680485185e-17
10.325707414425347 	 0.541311664415742 	 1.140836248210249e-17
9.325707414425347 	 0.5976303078076515 	 3.453145623158873e-17
8.325707414425347 	 0.6670418762840838 	 3.7429950401837766e-17
7.325707414425347 	 0.7547193781175311 	 3.843710783375046e-17
6.325707414425347 	 0.8689797529637481 	 4.7697009830883764e-17
5.325707414425347 	 1.0241070526789309 	 4.192148702340308e-17
4.325707414425347 	 1.2468971279649803 	 8.062266159279859e-17
3.325707414425347 	 1.594301462446768 	 1.2813490069415586e-16
2.325707414425347 	 2.213124726732211 	 3.7604707972092113e-16
1.325707414425347 	 3.6414476395758366 	 0.0002698055026768034
0.3257074144253469 	 11.669681525331226 	 5

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: overflow encountered in double_scalars
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in subtract


# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [ ]:
#Python 3

def min_iter(A, b, show_plot=True, eps=1e-12, maxiter=100):
    m, n = A.shape 
    x0 = np.ones(n)
    tau_series = []
    
    for i in range(maxiter): 
        r = np.matmul(A, x0) - b
        temp = np.matmul(A, r)
        tau = np.matmul(r, temp) / np.linalg.norm(temp, ord=2)**2
        tau_series.append(tau)
        x = x0 - tau * r
        
        if np.linalg.norm(x - x0) < eps: 
            break
        
        x0 = x
    
    return x

Tests performed *(making use of the allclose NumPy function)*.

In [ ]:
n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

xx = np.linalg.solve(A, b)

In [ ]:
x = min_iter(A, b)

In [ ]:
np.testing.assert_allclose(x, xx)
np.testing.assert_allclose(A @ x, b)

In [ ]:
#Python 3

for k in range(25): 
    A1 = A + np.diagflat([-k] * n)
    B_norm = norm(A1)
    x = min_iter(A1, b, False)
    
    xx = np.linalg.solve(A1, b)
    dist = np.linalg.norm(x - xx)
    
    print(A1[0, 0], '\t', B_norm, '\t', dist)

15.373423707353538 	 0.33688318691473734 	 8.144759948869233e-14
14.373423707353538 	 0.3599186187045102 	 3.4556925013716674e-14
13.373423707353538 	 0.38633818171402223 	 1.001975880471273e-13
12.373423707353538 	 0.4169475032737339 	 6.196003025502912e-14
11.373423707353538 	 0.45283028862683194 	 5.43281602596032e-14
10.373423707353538 	 0.49547981843737987 	 5.956544659271125e-14
9.373423707353538 	 0.5470130066640051 	 9.59771146087566e-14
8.373423707353538 	 0.6105347157974735 	 2.0290546202160942e-13
7.373423707353538 	 0.6907921005349461 	 1.947472844359116e-13
6.373423707353538 	 0.7954308152405096 	 3.644813535038975e-13
5.373423707353538 	 0.9376202762223229 	 2.4199020926043006e-13
4.373423707353538 	 1.1421938096062156 	 4.181380273004674e-13
3.3734237073535382 	 1.462477199852301 	 8.966180150479402e-13
2.3734237073535382 	 2.039321827238908 	 1.8593449626163264e-12
1.3734237073535382 	 3.4341680304660547 	 7.561876060651771e-07
0.37342370735353825 	 47.48324124939881 	 